# Preliminaries

To use the Neptune SPARQL endpoint, open the SSH tunnel using

```
ssh neptune -N
```

which reads the `~/.ssh/config` file containing:

```
host neptune-demo
 ForwardAgent yes
 User ec2-user # when using Amazon Linux
 HostName <your-ec2-address>
 IdentitiesOnly yes
 IdentityFile ~/.ssh/<your-ec2-key-file>.pem
 LocalForward 8182 <cluster-endpoint-neptune>:8182
 ```
 
 This allows access to the server through port 8182. 
 
 NOTE: at least on my computer, the Terminal window does not produce a prompt after executing this command. However, it does seem to work, since I get a response when I issue this command in a different Terminal window:
 
```
curl https://triplestore1.cluster-cml0hq81gymg.us-east-1.neptune.amazonaws.com:8182/status
```

## Configuration

In [32]:
import requests
import json

endpoint = 'https://triplestore1.cluster-cml0hq81gymg.us-east-1.neptune.amazonaws.com:8182/sparql'

sparql_request_header = {
        'Accept' : 'application/json',
#        'Content-Type': 'application/sparql-query',
        'Content-Type': 'application/x-www-form-urlencoded'
    }

# Low level functions

def send_sparql_query(query_string):
    """Sends a SPARQL query to an endpoint URL. Argument is the query string, returns a list of results."""
    # You can delete the two print statements if the queries are short. However, for large/long queries,
    # it's good to let the user know what's going on.
    #print('querying SPARQL endpoint to acquire item metadata')
    #response = requests.post(endpoint, data=query_string.encode('utf-8'), headers=sparql_request_header)
    response = requests.post(endpoint, data=dict(query=query_string), headers=sparql_request_header)
    #print(response.text) # uncomment to view the raw response, e.g. if you are getting an error
    data = response.json()

    # Extract the values from the response JSON
    results = data['results']['bindings']
    
    #print('done retrieving data')
    #print(json.dumps(results, indent=2))
    
    return results

def extract_local_name(iri):
    """Extracts the local name part of an IRI, e.g. a qNumber from a Wikidata IRI. Argument is the IRI, returns the local name string."""
    # pattern is http://www.wikidata.org/entity/Q6386232
    pieces = iri.split('/')
    last_piece = len(pieces)
    return pieces[last_piece - 1]


# Scripts

## Find out what graphs are in the triplestore

In [14]:
query_string = '''select distinct ?graph where {
graph ?graph {?s ?o ?p.}
}'''

results = send_sparql_query(query_string)
for result in results:
    graph_iri = result['graph']['value']
    print(graph_iri)

http://AATOut_ContribRels
http://AATOut_OrderedCollections
http://nomenclature_2022-02-02
http://AATOut_2Terms
http://AATOut_SourceRels
http://AATOut_Sources
http://AATOut_SemanticLinks
http://AATOut_1Subjects
http://AATOut_Lang_sameAs
http://AATOut_Contribs
http://AATOut_LCSHAlignment
http://AATOut_Notations
http://AATOut_ScopeNotes
http://AATOut_HierarchicalRels
http://AATOut_AssociativeRels
http://AATOut_ObsoleteSubjects


## Query the Nomenclature thesaurus

In [38]:
descriptive_nouns = ['plate', 'burner', 'bowl', 'mug', 'jar', 'vase', 'figure', 'ferrule', 'amulet', 'dish', 'coin', 'cup', 'buckle']
noun = descriptive_nouns[5]
noun

'jar'

In [51]:
for noun in descriptive_nouns:
    test_label = noun.title() # Nomenclature labels have the first letters of their labels capitalized
    print(test_label)
    query_string = '''prefix skos: <http://www.w3.org/2004/02/skos/core#>
    select distinct ?iri ?label 
    from <http://nomenclature_2022-02-02>
    where {
    {?iri skos:prefLabel "''' + test_label + '''"@en.}
    union
    {?iri skos:altLabel "''' + test_label + '''"@en.}

    #?iri skos:prefLabel "''' + test_label + '''"@en.
    }'''

    #print(query_string)

    results = send_sparql_query(query_string)
    found = False
    for result in results:
        iri = result['iri']['value']
        nomenclature_id = extract_local_name(iri)
        found = True
        print(nomenclature_id)
    if not found:
        print('no match')
    print()


Plate
no match

Burner
no match

Bowl
11792

Dlkjs
no match

Mug
4829

Jar
11795

Vase
1508

Figure
no match

Ferrule
6581

Amulet
13970

Dish
no match

Coin
13826

Cup
4771

Buckle
6562



In [53]:
for noun in descriptive_nouns:
    print(noun)
    query_string = '''prefix skos: <http://www.w3.org/2004/02/skos/core#>
    prefix skosxl: <http://www.w3.org/2008/05/skos-xl#>
    select distinct ?iri ?labelObject 
    from <http://AATOut_2Terms>
    where {
    {?iri skosxl:prefLabel ?labelObject.}
    union
    {?iri skosxl:altLabel ?labelObject.}
    ?labelObject skosxl:literalForm "''' + noun +'''"@en.
    }'''

    #print(query_string)

    results = send_sparql_query(query_string)

    found = False
    for result in results:
        iri = result['iri']['value']
        getty_id = extract_local_name(iri)
        found = True
        print(getty_id)
    if not found:
        print('no match')
    print()


plate
no match

burner
no match

bowl
no match

dlkjs
no match

mug
no match

jar
300195347

vase
300132254

figure
no match

ferrule
300233319

amulet
300266585

dish
no match

coin
no match

cup
no match

buckle
no match

